# EXCEED Python Qualtrics Survey - Detailed Analysis

This notebook provides a detailed analysis of the EXCEED Python Qualtrics survey data. It includes data loading, cleaning, and various analyses to understand the survey responses better. The ultimate goal of this analysis is to find the best subset of questions that can best differentiate between Python _beginner_/_novice_ and _advanced_/_expert_ users.

The Qualtrics survey is composed of the following blocks, where each participant is required to answer all questions in each block. Note that for blocks 3 to 8, each respondent is given a random subset of 2 out of the 7 available questions, hence the total number of survey-related questions is 16. The blocks are as follows:
1. **Consent Form**: Participants agree to take part in the survey. If they do not agree, they are redirected to the end of the survey.
2. **Self-assessment**:
    1. **Python Experience**: Participants self-assess their Python experience using Dreyfus levels.
    2. **Python Programming YoE**: Participants indicate their years of experience with Python programming.
    3. **General Programming YoE**: Participants indicate their years of experience with programming in general.
3. **General Programming Error Understanding**: 7 questions
4. **Python-Specific General Error Understanding**: 7 questions
5. **Code Reading / Understanding**: 7 questions
6. **Error Message Comprehension**: 7 questions
7. **Error Resolution**: 7 questions
8. **Error Message Comprehension**: 7 questions
9. **Natural Language Scenarios**: 7 questions
10. **Miscellanous Questions - Various Complexity & Scope**: 7 questions
11. **Self-Assessment of Results**: Participants self-assess the number of questions they answered correctly in the survey (0 to 16).

## Step 0: Install Required Libraries

Prior to running the analysis, ensure that the required libraries are installed. For this, you need only install the libraries defined in the `requirements.txt` file. You can do this by running the following command in your terminal:

```bash
pip install -r requirements.txt
```

In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Step 1: Load the data

For this step, we need to load 2 CSV files:
1. `survey_results.csv`: Contains the survey responses and some other metadata
2. `survey_answers.csv`: Contains the correct answers for the survey questions

The default files are in the same directory as this notebook. If you have the files in a different directory, please change the file paths accordingly.

In [2]:
import pandas as pd

file_path_survey_results = "../data/survey_results.csv"
file_path_survey_answers = "../data/survey_answers.csv"

# Step 0. Load data (files are semicolon‑delimited)
df_results = pd.read_csv(file_path_survey_results, sep=';')
df_answers = pd.read_csv(file_path_survey_answers, sep=';')

## Step 1: Build Answers DataFrame

To calculate the accuracy of the answers, we need to build a DataFrame that contains the correct answers for each question. This DataFrame will be used to compare the participants' responses against the correct answers.

In [3]:
answer_key = dict(zip(df_answers["QuestionId"], df_answers["CorrectAnswer"]))

# 1. Build columns for each question
question_cols = [q for q in answer_key if q in df_results.columns]

## Step 2: Mark Questions as Correct or Incorrect

To determine whether a participant answered a question correctly, we will compare their response to the correct answer. We will create a new DataFrame that contains the correctness of each answer (True for correct, False for incorrect).

In [4]:
# 2. Create a new DataFrame with the correct answers
correct_df = df_results[question_cols].eq(pd.Series(answer_key))
df_results["total_correct"] = correct_df.sum(axis=1)

## Step 3: Compute Accuracy for Each Question

Compute the accuracy for each question by calculating the percentage of correct answers among all participants who answered that question. This will help us identify which questions are more effective in differentiating between beginner/novice and advanced/expert users.

In [5]:
# Boolean mask: True if answered AND correct, NaN if unanswered
correct_mask = (
    df_results[question_cols]
        .eq(pd.Series(answer_key))
        .where(df_results[question_cols].notna())
)

correct_mask = correct_mask.astype("float")

# Per-question counts
correct_cnt  = correct_mask.sum()
answered_cnt = df_results[question_cols].notna().sum()

# Calculate the percentage of correct answers for each question
perc_correct = (correct_cnt / answered_cnt * 100).round(1).rename("% correct")

# Create a DataFrame with the percentage of correct answers
question_perf = (
    perc_correct.reset_index()
        .rename(columns={"index": "Question"})
        .sort_values("% correct", ascending=False)
        .reset_index(drop=True)
)

from IPython.display import display
display(question_perf)

,Question,% correct
0,Q8.7,100.0
1,Q7.4,100.0
2,Q10.6,100.0
3,Q10.4,100.0
4,Q3.5,100.0
5,Q7.5,100.0
6,Q6.3,100.0
7,Q4.1,100.0
8,Q6.1,100.0
9,Q8.3,100.0


## Step 4: Find Best Subset of Questions (2 per Block)

We will find the top 2 "best" questions from each block based on their accuracy. The criteria for "best" is a combination of item difficulty and item discrimination, which can be calculated using the following formula:

1. **Item Difficulty**: The percentage of correct answers for a given question.
2. **Item Discrimination**: The correlation with a respondent's overall ability.

In [6]:
from scipy.stats import pointbiserialr

# Total score per respondent (all questions from block 3 to block 10 inclusive); NaN treated as 0
df_results["total_score"] = correct_mask.fillna(0).sum(axis=1)

# Compute p (proportion correct) and r_pb (discrimination index)
records = []

for q in correct_mask.columns:
     # Boolean / NaN vector for this item
    item = correct_mask[q]
    
    # Skip items with < 10 valid answers to avoid unstable r
    if item.notna().sum() < 10:
        continue
    
    # Difficulty p
    p = item.mean(skipna=True)
    
    # Rest-of-test score (subtract the item itself if answered)
    rest = df_results["total_score"] - item.fillna(0)
    
    # Point-biserial correlation (returns r, p-value)
    r_pb, _ = pointbiserialr(item.dropna(), rest.loc[item.notna()])
    
    block = q.split(".")[0]
    records.append(
        {
            "block": block,
            "question": q,
            "p": p,
            "r_pb": r_pb
        }
    )

stats_df = pd.DataFrame(records)

# Add a numeric block column
stats_df["block_num"] = stats_df["block"].str.slice(1).astype(int)

# Keep items in an acceptable difficulty range (0.25 <= p <= 0.85)
stats_df = stats_df.query("0.25 <= p <= 0.85")

# Rank by discrimination index (r_pb) and take the top 2 per block
stats_df = (
    stats_df.sort_values(["block_num", "r_pb"], ascending=[True, False])
            .assign(rank=lambda d: d.groupby("block_num").cumcount() + 1)
)

top2_df = (
    stats_df[stats_df["rank"] <= 2]
            .sort_values(["block_num", "r_pb"], ascending=[True, False])
)
# Display the top 2 items per block
from IPython.display import display
display(top2_df[["block", "question", "p", "r_pb"]]
         .style.format({"p": "{:.2f}", "r_pb": "{:.2f}"}))

/Users/amoraru/Documents/GitHub/EXCEED/exceed-python-skill-assessment/.venv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:5405: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)


,block,question,p,r_pb
3,Q3,Q3.4,0.75,0.23
2,Q3,Q3.3,0.50,0.17
12,Q4,Q4.6,0.82,0.39
10,Q4,Q4.4,0.59,0.36
15,Q5,Q5.2,0.82,0.65
18,Q5,Q5.5,0.75,0.21
22,Q6,Q6.2,0.65,0.42
27,Q6,Q6.7,0.44,-0.01
29,Q7,Q7.2,0.56,0.58
30,Q7,Q7.3,0.75,0.38


## Step 5: Compute Average Correct Overall

Compute the average number of correct answers across all participants. This will give us an overall sense of how well participants performed on the survey.

In [7]:
avg_total_correct = df_results["total_correct"].mean()

## Step 6: Compute Average Correct by Block

For each of the actual survey blocks that contain skill-level assessment questions, we will compute the average correctness of the answers. This will help us understand how well participants performed in each block.

In [8]:
block_avgs = {}
for blk in range(3, 11):
    blk_cols = [c for c in question_cols if c.startswith(f"Q{blk}.")]
    block_avgs[f"Q{blk}"] = correct_df[blk_cols].sum(axis=1).mean()

## Step 7: Build Result Summary DataFrame

This DataFrame will summarize the results of the analysis, including the average correctness for each question, the overall average correctness, and the average correctness by block. This will provide a comprehensive overview of the survey results.

In [9]:
# Collect rows for the summary report
rows = [
    ("Average correct answers (all blocks)", avg_total_correct),
]

# Block averages (Q3 to Q10)
rows.extend(
    [(f"Average correct answers - Block {blk}", val) for blk, val in block_avgs.items()]
)

# Compute correlations
rows.extend([
    ("Correlation: Python YoE with total correct", df_results[["Q2.2", "total_correct"]].corr().iloc[0, 1]),
    ("Correlation: General YoE with total correct", df_results[["Q2.3", "total_correct"]].corr().iloc[0, 1]),
    ("Correlation: Total correct answers with self-reported correct answers", df_results["total_correct"].corr(df_results["Q11.1"]))
])

## Step 8: Display Result Summary

Finally, we will display the result summary DataFrame, which contains the average correctness of answers for each block and the overall correctness, as well as some additional correlation metrics.

In [10]:
from IPython.display import display

# Create a DataFrame for the summary report
summary_df = pd.DataFrame(rows, columns=["Metric", "Value"])

# Display the summary report
display(summary_df.style.set_caption("Qualtrics Python Skill Level Assessment - Correctness Metrics").format(precision=2))

,Metric,Value
0,Average correct answers (all blocks),11.06
1,Average correct answers - Block Q3,1.46
2,Average correct answers - Block Q4,1.40
3,Average correct answers - Block Q5,1.41
4,Average correct answers - Block Q6,1.37
5,Average correct answers - Block Q7,1.34
6,Average correct answers - Block Q8,1.43
7,Average correct answers - Block Q9,1.30
8,Average correct answers - Block Q10,1.34
9,Correlation: Python YoE with total correct,0.25
